# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the pipeline, we will set up an objective function based on some assumptions about our business. 

In [1]:
import evalml
from evalml.objectives import FraudDetection
from evalml.preprocessing import split_data

## Load Transactions

In [2]:
X, y = evalml.demos.load_fraud()

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992

Labels
False    84.82%
True     15.18%
Name: fraud, dtype: object


## Define Objective

To make optimize the pipelines toward your specific business needs, you can set your own assumptions for the cost of fraud. These parameters are

* `retry_percentage` - what percentage of customers will retry a transaction if it is declined?
* `interchange_fee` - how much of each successfull transaction do you collect?
* `fraud_payout_percentage` - how percentage of fraud will you be unable to collect

Using these parameters, EvalML determines attempt to build a pipeline that will minimize the financial loss.

In [3]:
fraud_objective = FraudDetection(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount'  # column in data that contains the amount of the transaction
)

## Search for best modeling pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [4]:
# select numeric data before running AutoClassifer
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_time=120,
                            max_pipelines=10)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Detection. Lower score is better.

Searching up to 10 pipelines. Will stop searching for new pipelines after 120 seconds.

Possible model types: random_forest, linear_model, xgboost

Testing Random Forest w/ imputation:  80%|████████  | 8/10 [02:05<00:37, 18.68s/it]               

Max time elapsed. Stopping search early.
Testing Random Forest w/ imputation:  80%|████████  | 8/10 [02:05<00:31, 15.65s/it]

✔ Optimization finished


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the objective function we defined

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,3,LogisticRegressionPipeline,2575489.89,False,"{'penalty': 'l2', 'C': 6.239401330891865, 'imp..."
1,0,LogisticRegressionPipeline,2605535.77,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
2,1,RFClassificationPipeline,10307633.90,False,"{'n_estimators': 569, 'max_depth': 630, 'imput..."
3,7,RFClassificationPipeline,10623222.35,False,"{'n_estimators': 715, 'max_depth': 487, 'imput..."
4,5,RFClassificationPipeline,10982475.12,False,"{'n_estimators': 609, 'max_depth': 71, 'impute..."
5,2,RFClassificationPipeline,11610654.75,False,"{'n_estimators': 369, 'max_depth': 10, 'impute..."
6,4,XGBoostPipeline,11610654.75,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
7,6,XGBoostPipeline,11610654.75,False,"{'eta': 0.38438170729269994, 'min_child_weight..."


to select the best pipeline we can run

In [7]:
best_pipeline = clf.best_pipeline

to select another pipeline we can use the id

In [8]:
pipeline = clf.get_pipeline(0)

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [9]:
clf.describe_pipeline(0)

************************
* Pipeline Description *
************************

Pipeline Name: LogisticRegression w/ imputation + scaling
Model type: linear_model
Objective: Fraud Detection (lower is better)
Total training time (including CV): 4.6 seconds

Parameters
• penalty: l2
• C: 8.444214828324364
• impute_strategy: most_frequent

Cross Validation
        F1  Precision  Recall   AUC  Log Loss  Fraud Detection
0    0.269      0.155   0.269 0.514    28.161      2566886.980
1    0.271      0.157   0.271 0.521    27.643      2605535.770
2    0.264      0.152   0.264 0.500    29.270      2507515.970
mean 0.268      0.155   0.268 0.512    28.358      2559979.573
std  0.004      0.003   0.004 0.010     0.831        49373.621


## Test on hold out

Finally, we retrain the best pipeline on all of the training data

In [10]:
best_pipeline.fit(X_train, y_train)

Now, we can score the pipeline on the hold out data. 

In [11]:
best_pipeline.score(X_holdout, y_holdout)

30542872.849999994